In [7]:
# PARAMETER
#-------------
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False
try: 
    IS_MASTER
except: 
    IS_MASTER = False

if IS_MASTER:
    '''
    Declare master variable here
    '''
    DATA_DIR = './data/big_data/'
    # RAW_FILE = f'/path/to/smallData.csv' # use "small data" here
    # PROCESSED_FILE = f'{DATA_DIR}processed.pkl' # always use pickle for fast I/O!
    # OTHER_PREPROCESS_PARAMETER = ... # e.g. batch size, sliding window size, etc
    
    print("[MASTER] using big dataset...")
else:
    # Testing dataset to construct code
    DATA_DIR = './data/small_data/'
    print("[NOT MASTER] using small dataset...")
    

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(DATA_DIR))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
    
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")